In [ ]:
import pandas as pd

In [ ]:
result_df = pd.read_parquet('/content/sleep_result_44659288-6985ad26-e34f-4a9e-bccc-7f3d326b8b72.parquet')
result_df = result_df.dropna(axis=1, how='all')
result_df.head(3)

In [ ]:
sample_df = result_df.head(3)
sample_df.to_csv('/content/sample_result.csv', index=False)

In [ ]:
for col in result_df.columns:
    print(col)

In [ ]:
score_df = pd.read_parquet('/content/sleep_score_44659288-1638f8a8-b506-406b-9143-857be7e41a68.parquet')
score_df = score_df.dropna(axis=1, how='all')
score_df.head(3)

In [ ]:
sample_df = score_df.head(3)
sample_df.to_csv('/content/sample_score.csv', index=False)

In [ ]:
for col in score_df.columns:
    print(col)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Load data
result_df = pd.read_parquet('/content/sleep_result_44659288-6985ad26-e34f-4a9e-bccc-7f3d326b8b72.parquet')
result_df = result_df.dropna(axis=1, how='all')

score_df = pd.read_parquet('/content/sleep_score_44659288-1638f8a8-b506-406b-9143-857be7e41a68.parquet')
score_df = score_df.dropna(axis=1, how='all')

# Merge datasets
df = pd.merge(result_df, score_df, on='night', how='inner')

# ============================================================================
# 1. DATA PREPROCESSING
# ============================================================================

# Parse datetime columns with mixed format support
df['sleep_start'] = pd.to_datetime(df['sleepResult_hypnogram_sleepStart'], format='mixed', utc=True)
df['sleep_end'] = pd.to_datetime(df['sleepResult_hypnogram_sleepEnd'], format='mixed', utc=True)

# Convert to local timezone (assuming Asia/Ho_Chi_Minh based on +08:00/+09:00 in data)
df['sleep_start'] = df['sleep_start'].dt.tz_convert('Asia/Ho_Chi_Minh')
df['sleep_end'] = df['sleep_end'].dt.tz_convert('Asia/Ho_Chi_Minh')

# Extract wake time (hour)
df['wake_hour'] = df['sleep_end'].dt.hour + df['sleep_end'].dt.minute/60

# Extract bedtime (hour)
df['bedtime_hour'] = df['sleep_start'].dt.hour + df['sleep_start'].dt.minute/60

# Parse duration strings (PT5H6M30S format)
def parse_duration(duration_str):
    if pd.isna(duration_str):
        return np.nan
    try:
        duration_str = str(duration_str)
        hours = 0
        minutes = 0
        seconds = 0

        # Extract hours
        if 'H' in duration_str:
            hours = int(duration_str.split('H')[0].split('T')[1])
            duration_str = duration_str.split('H')[1]  # Remove hours part
        else:
            duration_str = duration_str.split('T')[1]  # Remove PT prefix

        # Extract minutes
        if 'M' in duration_str:
            minutes = int(duration_str.split('M')[0])
            duration_str = duration_str.split('M')[1]  # Remove minutes part

        # Extract seconds
        if 'S' in duration_str:
            seconds = float(duration_str.replace('S', ''))

        return hours + minutes/60 + seconds/3600
    except:
        return np.nan

df['total_sleep_hours'] = df['evaluation_asleepDuration'].apply(parse_duration)
df['time_in_bed_hours'] = df['evaluation_sleepSpan'].apply(parse_duration)
df['interruption_duration'] = df['evaluation_interruptions_totalDuration'].apply(parse_duration)

# Parse sleep stages
df['wake_hours'] = df['evaluation_phaseDurations_wake'].apply(parse_duration)
df['rem_hours'] = df['evaluation_phaseDurations_rem'].apply(parse_duration)
df['light_hours'] = df['evaluation_phaseDurations_light'].apply(parse_duration)
df['deep_hours'] = df['evaluation_phaseDurations_deep'].apply(parse_duration)

# Day of week
df['day_of_week'] = df['sleep_end'].dt.day_name()

# Calculate if woke up close to 5am (within 30 min)
df['woke_near_5am'] = ((df['wake_hour'] >= 4.5) & (df['wake_hour'] <= 5.5))

print("\n" + "="*80)
print("SLEEP OPTIMIZATION REPORT: 5AM WAKE-UP GOAL")
print("="*80)
print(f"\n📊 DATA OVERVIEW")
print(f"   Period: {df['sleep_end'].min().date()} to {df['sleep_end'].max().date()}")
print(f"   Total nights: {len(df)} | Successful 5am wake-ups: {df['woke_near_5am'].sum()} ({df['woke_near_5am'].mean()*100:.1f}%)")

# ============================================================================
# 2. KEY METRICS FOR 5AM WAKE-UP
# ============================================================================

print(f"\n" + "-"*80)
print("YOUR 5AM WAKE-UP PATTERN")
print("-"*80)

# Filter for 5am wake-ups
df_5am = df[df['woke_near_5am']]

if len(df_5am) > 0:
    optimal_bedtime = df_5am['bedtime_hour'].median()
    bedtime_display = f"{int(optimal_bedtime)}:{int((optimal_bedtime%1)*60):02d}"

    print(f"\n✅ SUCCESS FORMULA (based on {len(df_5am)} successful nights):")
    print(f"   • Bedtime: {bedtime_display} (range: {df_5am['bedtime_hour'].min():.1f}-{df_5am['bedtime_hour'].max():.1f}h)")
    print(f"   • Sleep duration: {df_5am['total_sleep_hours'].median():.1f}h (range: {df_5am['total_sleep_hours'].min():.1f}-{df_5am['total_sleep_hours'].max():.1f}h)")
    print(f"   • Sleep efficiency: {df_5am['evaluation_analysis_efficiencyPercent'].median():.1f}%")
    print(f"   • Sleep score: {df_5am['sleepScoreResult_sleepScore'].median():.1f}/100")
    print(f"   • Deep sleep: {df_5am['deep_hours'].median():.1f}h ({df_5am['evaluation_phaseDurations_deepPercentage'].median():.1f}%)")
    print(f"   • REM sleep: {df_5am['rem_hours'].median():.1f}h ({df_5am['evaluation_phaseDurations_remPercentage'].median():.1f}%)")

    print(f"\n📊 COMPARISON TO YOUR AVERAGE:")
    print(f"   • Avg wake time: {df['wake_hour'].mean():.1f}:00 ({(df['wake_hour'].mean()-5)*60:.0f} min from 5am target)")
    print(f"   • Avg sleep: {df['total_sleep_hours'].mean():.1f}h (5am nights: {df_5am['total_sleep_hours'].mean():.1f}h)")
    print(f"   • Avg efficiency: {df['evaluation_analysis_efficiencyPercent'].mean():.1f}% (5am: {df_5am['evaluation_analysis_efficiencyPercent'].mean():.1f}%)")
else:
    print(f"\n⚠️  No successful 5am wake-ups in dataset")
    print(f"   Current avg wake time: {df['wake_hour'].mean():.1f}:00")
    print(f"   Suggested bedtime for 7h sleep: 22:00")
    print(f"   Suggested bedtime for 6h sleep: 23:00")

# Convert to local timezone (assuming Asia/Ho_Chi_Minh based on +08:00/+09:00 in data)
df['sleep_start'] = df['sleep_start'].dt.tz_convert('Asia/Ho_Chi_Minh')
df['sleep_end'] = df['sleep_end'].dt.tz_convert('Asia/Ho_Chi_Minh')

# Extract wake time (hour)
df['wake_hour'] = df['sleep_end'].dt.hour + df['sleep_end'].dt.minute/60

# Extract bedtime (hour)
df['bedtime_hour'] = df['sleep_start'].dt.hour + df['sleep_start'].dt.minute/60

# Parse duration strings (PT5H6M30S format)
def parse_duration(duration_str):
    if pd.isna(duration_str):
        return np.nan
    try:
        duration_str = str(duration_str)
        hours = 0
        minutes = 0
        seconds = 0

        # Extract hours
        if 'H' in duration_str:
            hours = int(duration_str.split('H')[0].split('T')[1])
            duration_str = duration_str.split('H')[1]  # Remove hours part
        else:
            duration_str = duration_str.split('T')[1]  # Remove PT prefix

        # Extract minutes
        if 'M' in duration_str:
            minutes = int(duration_str.split('M')[0])
            duration_str = duration_str.split('M')[1]  # Remove minutes part

        # Extract seconds
        if 'S' in duration_str:
            seconds = float(duration_str.replace('S', ''))

        return hours + minutes/60 + seconds/3600
    except:
        return np.nan

df['total_sleep_hours'] = df['evaluation_asleepDuration'].apply(parse_duration)
df['time_in_bed_hours'] = df['evaluation_sleepSpan'].apply(parse_duration)
df['interruption_duration'] = df['evaluation_interruptions_totalDuration'].apply(parse_duration)

# Parse sleep stages
df['wake_hours'] = df['evaluation_phaseDurations_wake'].apply(parse_duration)
df['rem_hours'] = df['evaluation_phaseDurations_rem'].apply(parse_duration)
df['light_hours'] = df['evaluation_phaseDurations_light'].apply(parse_duration)
df['deep_hours'] = df['evaluation_phaseDurations_deep'].apply(parse_duration)

# Day of week
df['day_of_week'] = df['sleep_end'].dt.day_name()

# Calculate if woke up close to 5am (within 30 min)
df['woke_near_5am'] = ((df['wake_hour'] >= 4.5) & (df['wake_hour'] <= 5.5))

print(f"\n" + "-"*80)
print("TOP INSIGHTS")
print("-"*80)

# Find strongest correlations with sleep quality
quality_metrics = ['total_sleep_hours', 'bedtime_hour', 'evaluation_analysis_efficiencyPercent',
                   'evaluation_analysis_continuityIndex', 'deep_hours', 'rem_hours']
correlations = df[quality_metrics + ['sleepScoreResult_sleepScore']].corr()['sleepScoreResult_sleepScore'].sort_values(ascending=False)

print(f"\n🎯 WHAT DRIVES YOUR SLEEP QUALITY:")
top_3 = correlations[1:4]
for metric, corr in top_3.items():
    metric_name = metric.replace('evaluation_analysis_', '').replace('_', ' ').title()
    print(f"   • {metric_name}: {corr:+.2f} correlation")

# Best bedtime window analysis
bedtime_quality = df.groupby(pd.cut(df['bedtime_hour'], bins=[20, 22, 23, 24, 26])).agg({
    'sleepScoreResult_sleepScore': 'mean',
    'total_sleep_hours': 'mean',
    'woke_near_5am': 'sum'
}).round(1)

print(f"\n🛏️  BEST BEDTIME WINDOWS:")
best_window = bedtime_quality['sleepScoreResult_sleepScore'].idxmax()
print(f"   Highest quality: {best_window} (score: {bedtime_quality.loc[best_window, 'sleepScoreResult_sleepScore']:.1f})")

# Day of week pattern
dow_analysis = df.groupby('day_of_week')['sleepScoreResult_sleepScore'].mean().sort_values(ascending=False)
print(f"\n📅 YOUR BEST SLEEP DAYS:")
for day in dow_analysis.head(3).index:
    print(f"   • {day}: {dow_analysis[day]:.1f} avg score")

print(f"\n" + "-"*80)
print("ACTION PLAN")
print("-"*80)

if len(df_5am) > 0:
    optimal_bedtime = df_5am['bedtime_hour'].median()
    bedtime_display = f"{int(optimal_bedtime)}:{int((optimal_bedtime%1)*60):02d}"

    print(f"\n1. 🎯 TARGET BEDTIME: {bedtime_display}")
    print(f"   Based on your successful 5am wake-ups")

    print(f"\n2. ⏱️  SLEEP DURATION: {df_5am['total_sleep_hours'].median():.1f} hours")
    print(f"   Range that works: {df_5am['total_sleep_hours'].quantile(0.25):.1f}-{df_5am['total_sleep_hours'].quantile(0.75):.1f}h")

    print(f"\n3. 📊 QUALITY BENCHMARKS:")
    print(f"   • Efficiency: >{df_5am['evaluation_analysis_efficiencyPercent'].quantile(0.5):.0f}%")
    print(f"   • Deep sleep: >{df_5am['deep_hours'].quantile(0.5):.1f}h")
    print(f"   • REM sleep: >{df_5am['rem_hours'].quantile(0.5):.1f}h")

    print(f"\n4. ⚠️  RED FLAGS TO AVOID:")
    worst_nights = df.nsmallest(10, 'sleepScoreResult_sleepScore')
    print(f"   • Bedtime later than {worst_nights['bedtime_hour'].median():.1f}:00")
    print(f"   • More than {worst_nights['evaluation_interruptions_totalCount'].median():.0f} interruptions")
    print(f"   • Less than {worst_nights['deep_hours'].median():.1f}h deep sleep")
else:
    print(f"\n1. 🎯 START WITH: 22:00-23:00 bedtime")
    print(f"   For 6-7 hours sleep before 5am")

    print(f"\n2. 📊 TARGET QUALITY:")
    print(f"   • Efficiency: >90%")
    print(f"   • Deep sleep: >1h")
    print(f"   • Sleep score: >60")

print(f"\n5. 🔄 CONSISTENCY IS KEY:")
print(f"   • Stick to ±30min of target bedtime")
print(f"   • Even on weekends")
print(f"   • Track for 2 weeks, then adjust")

print("\n" + "="*80)
print("GENERATING VISUALIZATIONS...")
print("="*80 + "\n")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Sleep Analysis: Optimizing for 5AM Wake-Up', fontsize=14, fontweight='bold')

# 1. Bedtime vs Wake Time with 5am target
scatter = axes[0, 0].scatter(df['bedtime_hour'], df['wake_hour'],
                             c=df['sleepScoreResult_sleepScore'],
                             cmap='RdYlGn', s=40, alpha=0.6, edgecolors='black', linewidth=0.5)
axes[0, 0].axhline(5, color='red', linestyle='--', linewidth=2, label='5am target', alpha=0.7)
axes[0, 0].set_xlabel('Bedtime (hour)', fontsize=10)
axes[0, 0].set_ylabel('Wake Time (hour)', fontsize=10)
axes[0, 0].set_title('Bedtime → Wake Time Pattern', fontsize=11, fontweight='bold')
axes[0, 0].set_xlim(20, 26)
axes[0, 0].set_xticks([20, 21, 22, 23, 24, 25, 26])
axes[0, 0].set_xticklabels(['8pm', '9pm', '10pm', '11pm', '12am', '1am', '2am'])
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)
plt.colorbar(scatter, ax=axes[0, 0], label='Sleep Score')

# 2. Sleep duration distribution with 5am success
axes[0, 1].hist(df['total_sleep_hours'], bins=25, color='steelblue', alpha=0.6, edgecolor='black', label='All nights')
if len(df_5am) > 0:
    axes[0, 1].hist(df_5am['total_sleep_hours'], bins=15, color='green', alpha=0.7, edgecolor='black', label='5am wake-ups')
axes[0, 1].axvline(df['total_sleep_hours'].mean(), color='blue', linestyle='--', linewidth=2, alpha=0.5, label='Avg')
axes[0, 1].set_xlabel('Total Sleep (hours)', fontsize=10)
axes[0, 1].set_ylabel('Frequency', fontsize=10)
axes[0, 1].set_title('Sleep Duration Distribution', fontsize=11, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3, axis='y')

# 3. Sleep stages comparison
if len(df_5am) > 0:
    stages_all = [df['light_hours'].mean(), df['deep_hours'].mean(), df['rem_hours'].mean(), df['wake_hours'].mean()]
    stages_5am = [df_5am['light_hours'].mean(), df_5am['deep_hours'].mean(), df_5am['rem_hours'].mean(), df_5am['wake_hours'].mean()]

    x = np.arange(4)
    width = 0.35

    axes[1, 0].bar(x - width/2, stages_all, width, label='All nights', color='steelblue', alpha=0.7)
    axes[1, 0].bar(x + width/2, stages_5am, width, label='5am wake-ups', color='green', alpha=0.7)
    axes[1, 0].set_ylabel('Hours', fontsize=10)
    axes[1, 0].set_title('Sleep Stages Comparison', fontsize=11, fontweight='bold')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(['Light', 'Deep', 'REM', 'Wake'])
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3, axis='y')
else:
    stage_data = [df['light_hours'].mean(), df['deep_hours'].mean(), df['rem_hours'].mean(), df['wake_hours'].mean()]
    axes[1, 0].bar(['Light', 'Deep', 'REM', 'Wake'], stage_data, color=['#66b3ff', '#99ff99', '#ffcc99', '#ff9999'])
    axes[1, 0].set_ylabel('Hours', fontsize=10)
    axes[1, 0].set_title('Average Sleep Stages', fontsize=11, fontweight='bold')
    axes[1, 0].grid(alpha=0.3, axis='y')

# 4. Efficiency trend over time
df_sorted = df.sort_values('sleep_end')
axes[1, 1].plot(df_sorted['sleep_end'], df_sorted['evaluation_analysis_efficiencyPercent'],
                marker='o', markersize=3, linestyle='-', alpha=0.4, color='steelblue')
# Add rolling average
df_sorted['efficiency_ma'] = df_sorted['evaluation_analysis_efficiencyPercent'].rolling(window=7).mean()
axes[1, 1].plot(df_sorted['sleep_end'], df_sorted['efficiency_ma'],
                color='red', linewidth=2, label='7-day avg')
axes[1, 1].axhline(df['evaluation_analysis_efficiencyPercent'].mean(),
                   color='green', linestyle='--', linewidth=2, alpha=0.5, label='Overall avg')
axes[1, 1].set_xlabel('Date', fontsize=10)
axes[1, 1].set_ylabel('Sleep Efficiency (%)', fontsize=10)
axes[1, 1].set_title('Sleep Efficiency Over Time', fontsize=11, fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=30)
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('sleep_analysis_5am.png', dpi=300, bbox_inches='tight')

print("✅ Analysis complete! Visualization saved as 'sleep_analysis_5am.png'\n")
print("="*80)